In [24]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [25]:
ori_data = np.load('../Data/samples_24/fmri_norm_truth.npy')
fake_data = np.load('../Data/ldm_fake_fmri_100_16_12_2.npy')
iterations = 5

In [26]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.10762974345382882 

Iter 1:  context-fid = 0.14148712048421522 

Iter 2:  context-fid = 0.15309146421403266 

Iter 3:  context-fid = 0.13544819506268782 

Iter 4:  context-fid = 0.15854954190322487 

Final Score:  0.13924121302359788 ± 0.02470378690745372


In [27]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 1.0486709237187344 

Iter 1:  cross-correlation = 1.0046157430223552 

Iter 2:  cross-correlation = 1.0303409921309434 

Iter 3:  cross-correlation = 1.0397769231840122 

Iter 4:  cross-correlation = 1.0573118744690277 

Final Score:  1.0361432913050144 ± 0.02518701528584536


In [28]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [29]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

training: 100%|██████████| 2000/2000 [00:26<00:00, 75.50it/s]


Iter 0:  0.08942885771543085 , 0.8597194388777555 , 0.3191382765531062 



training: 100%|██████████| 2000/2000 [00:26<00:00, 75.15it/s]


Iter 1:  0.1207414829659319 , 0.5435871743486974 , 0.6978957915831663 



training: 100%|██████████| 2000/2000 [00:26<00:00, 75.81it/s]


Iter 2:  0.08842685370741488 , 0.7760521042084169 , 0.40080160320641284 



training: 100%|██████████| 2000/2000 [00:26<00:00, 74.98it/s]


Iter 3:  0.11648296593186369 , 0.5696392785571143 , 0.6633266533066132 



training: 100%|██████████| 2000/2000 [00:26<00:00, 74.81it/s]


Iter 4:  0.09268537074148298 , 0.6052104208416834 , 0.5801603206412825 

sine:
Final Score:  0.10155310621242486 ± 0.01952442266699934



In [30]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [00:46<00:00, 107.01it/s]


0  epoch:  0.09858660452834246 



training: 100%|██████████| 5000/5000 [00:47<00:00, 106.37it/s]


1  epoch:  0.09889442893839807 



training: 100%|██████████| 5000/5000 [00:45<00:00, 108.83it/s]


2  epoch:  0.09895575990429527 



training: 100%|██████████| 5000/5000 [00:45<00:00, 110.53it/s]


3  epoch:  0.09872727292433577 



training: 100%|██████████| 5000/5000 [00:46<00:00, 107.18it/s]


4  epoch:  0.09887304065609358 

sine:
Final Score:  0.09880742139029303 ± 0.00018527896508916644

